# Acoustic Shield - Training & Deployment

This notebook trains and deploys the audio classification model using AWS SageMaker.

## Pipeline Overview
1. **Setup Configuration** - Define S3 paths, IAM role, and hyperparameters
2. **Create Training Job** - Fine-tune wav2vec2-base on audio data
3. **Deploy Endpoint** - Deploy real-time inference endpoint
4. **Test Endpoint** - Smoke test with sample audio

## Dataset Structure
Training data must be organized in audiofolder format:
```
s3://acousticshield-ml/train/
├── Normal/*.wav
├── TireSkid/*.wav
├── EmergencyBraking/*.wav
└── CollisionImminent/*.wav
```

## Audio Requirements
- **Format**: WAV (mono or stereo)
- **Sample Rate**: 16 kHz (auto-resampled if different)
- **Duration**: 1-5 seconds recommended

## Customization Options
- **Change epochs**: Modify `EPOCHS` parameter (default: 4)
- **Change learning rate**: Modify `LEARNING_RATE` (default: 3e-5)
- **Change batch size**: Modify `BATCH_SIZE` (default: 8)
- **Skip validation**: Set `VAL_S3 = None`

## Step 1: Configuration

⚠️ **IMPORTANT**: No region hardcoding - SageMaker auto-detects region from bucket location.

In [ ]:
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker.serializers import DataSerializer
from sagemaker.deserializers import JSONDeserializer
import json
from datetime import datetime

print("✓ Imports complete")

In [ ]:
# ============================================================================
# CONFIGURATION - Modify these parameters as needed
# ============================================================================

# S3 paths
TRAIN_S3 = "s3://acousticshield-ml/train_split/"     # Training data (80% split)
VAL_S3 = "s3://acousticshield-ml/val/"               # Validation data (20% split)
MODEL_OUTPUT_S3 = "s3://acousticshield-ml/models/"   # Model artifacts output

# IAM Role
ROLE_NAME = "role-sagemaker-train"                    # IAM role name (not ARN)

# Hyperparameters
EPOCHS = 1                 # Just 1 epoch for hackathon speed! (was 4)
LEARNING_RATE = 5e-5       # Faster convergence (was 3e-5)
BATCH_SIZE = 16            # Larger batches = faster (was 8)
WARMUP_STEPS = 50          # Less warmup = faster (was 500)
GRADIENT_ACCUMULATION = 1  # Gradient accumulation steps (increase for effective larger batch)

# Instance configuration - Try these in order if you hit quota limits:
# ⚠️ G5 instances often have 0 quota on new accounts - try G4DN first!
# RECOMMENDED ORDER (try from top to bottom):
#   "ml.g4dn.xlarge"    - NVIDIA T4 GPU, 16GB memory, ~$0.74/hour (MOST LIKELY to work!)
#   "ml.g4dn.2xlarge"   - NVIDIA T4 GPU, 32GB memory, ~$1.00/hour (more memory)
#   "ml.g4dn.4xlarge"   - NVIDIA T4 GPU, 64GB memory, ~$1.50/hour (even more memory)
# IF G4DN FAILS, try older generations:
#   "ml.p2.xlarge"      - NVIDIA K80 GPU, 61GB memory, ~$1.26/hour (older but usually available)
#   "ml.p3.2xlarge"     - NVIDIA V100 GPU, 61GB memory, ~$3.83/hour (expensive, may have quota)
# AVOID (usually 0 quota on new accounts):
#   "ml.g5.xlarge"      - Quota = 0 on most new accounts ❌
#   "ml.g5.2xlarge"     - Quota = 0 on most new accounts ❌
TRAIN_INSTANCE_TYPE = "ml.g4dn.xlarge"  # Change this if you get quota errors
TRAIN_INSTANCE_COUNT = 1                 # Number of training instances
ENDPOINT_INSTANCE_TYPE = "ml.m5.xlarge" # CPU instance for inference
ENDPOINT_INSTANCE_COUNT = 1              # Number of endpoint instances

# Model configuration
TRANSFORMERS_VERSION = "4.28"  # HuggingFace Transformers version (supports CPU)
PYTORCH_VERSION = "2.0"        # PyTorch version
PYTHON_VERSION = "py310"       # Python version

print("="*80)
print("Acoustic Shield - Training Configuration")
print("="*80)
print(f"Training data: {TRAIN_S3}")
print(f"Validation data: {VAL_S3 if VAL_S3 else 'None (will split from train)'}")
print(f"Model output: {MODEL_OUTPUT_S3}")
print(f"IAM Role: {ROLE_NAME}")
print(f"\nHyperparameters:")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Warmup Steps: {WARMUP_STEPS}")
print(f"\nInstances:")
print(f"  Training: {TRAIN_INSTANCE_TYPE} x {TRAIN_INSTANCE_COUNT}")
print(f"  Endpoint: {ENDPOINT_INSTANCE_TYPE} x {ENDPOINT_INSTANCE_COUNT}")
print("="*80)

## Step 2: Initialize SageMaker Session

Auto-detect region from S3 bucket location.

In [ ]:
# Auto-detect region from S3 bucket
s3 = boto3.client('s3')
bucket_name = TRAIN_S3.split('/')[2]  # Extract bucket from s3://bucket/path
bucket_location = s3.get_bucket_location(Bucket=bucket_name)['LocationConstraint']
region = bucket_location if bucket_location else 'us-east-1'

print(f"🌍 Detected region: {region}")

# Initialize boto3 session with detected region
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

# Get IAM role ARN
iam_client = boto_session.client('iam')
role_response = iam_client.get_role(RoleName=ROLE_NAME)
TRAIN_ROLE_ARN = role_response['Role']['Arn']

print(f"✓ SageMaker session initialized")
print(f"✓ Region: {region}")
print(f"✓ Role ARN: {TRAIN_ROLE_ARN}")

In [ ]:
# Emergency: Stop all in-progress training jobs
import boto3

sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

print("🔍 Looking for in-progress training jobs...")
response = sagemaker_client.list_training_jobs(
    StatusEquals='InProgress',
    MaxResults=10
)

if response['TrainingJobSummaries']:
    print(f"\n⚠️  Found {len(response['TrainingJobSummaries'])} in-progress job(s):\n")
    
    for job in response['TrainingJobSummaries']:
        job_name = job['TrainingJobName']
        status = job['TrainingJobStatus']
        created = job['CreationTime']
        
        print(f"📦 Job: {job_name}")
        print(f"   Status: {status}")
        print(f"   Created: {created}")
        
        # Stop the job
        print(f"   🛑 Stopping job...")
        try:
            sagemaker_client.stop_training_job(TrainingJobName=job_name)
            print(f"   ✅ Stop command sent successfully!\n")
        except Exception as e:
            print(f"   ❌ Error: {e}\n")
    
    print("⏳ Jobs are stopping... wait 1-2 minutes before starting new training.")
else:
    print("✅ No in-progress training jobs found!")
    print("   You're clear to start training!")

## ALTERNATIVE: Use Existing Trained Model (If Available)

⚡ **Skip Training**: If you already have a trained model in S3, deploy it directly!

This will check for existing model artifacts and deploy them without retraining.

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import boto3

print("? Checking for existing trained models in S3...\n")

# Check for existing model artifacts
s3_client = boto3.client('s3')
bucket = MODEL_OUTPUT_S3.split('/')[2]
prefix = '/'.join(MODEL_OUTPUT_S3.split('/')[3:])

try:
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=10)
    
    if 'Contents' in response:
        # Find model.tar.gz files
        model_files = [obj['Key'] for obj in response['Contents'] 
                      if obj['Key'].endswith('model.tar.gz')]
        
        if model_files:
            # Use the most recent model
            latest_model = sorted(model_files)[-1]
            model_data_s3 = f"s3://{bucket}/{latest_model}"
            
            print(f"✅ Found existing model!")
            print(f"📦 Model: {model_data_s3}\n")
            
            # Create model with your custom inference code
            huggingface_model = HuggingFaceModel(
                model_data=model_data_s3,
                role=TRAIN_ROLE_ARN,
                entry_point='inference.py',
                source_dir='../training',
                transformers_version=TRANSFORMERS_VERSION,
                pytorch_version=PYTORCH_VERSION,
                py_version=PYTHON_VERSION,
            )
            
            # Generate endpoint name
            timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
            endpoint_name = f'acousticshield-existing-{timestamp}'
            
            print(f"🌐 Endpoint: {endpoint_name}")
            print(f"💻 Instance: {ENDPOINT_INSTANCE_TYPE}")
            print(f"\n⏰ Deploying existing model... (5-8 minutes)\n")
            
            # Deploy endpoint
            predictor = huggingface_model.deploy(
                initial_instance_count=ENDPOINT_INSTANCE_COUNT,
                instance_type=ENDPOINT_INSTANCE_TYPE,
                endpoint_name=endpoint_name,
                serializer=DataSerializer(content_type='audio/wav'),
                deserializer=JSONDeserializer(),
            )
            
            print("\n" + "="*80)
            print("✅ EXISTING MODEL DEPLOYED!")
            print("="*80)
            print(f"🌐 Endpoint: {endpoint_name}")
            print(f"✅ Status: InService")
            print(f"? Model: {model_data_s3}")
            print(f"\n💡 Next: Run Step 6 (Test Endpoint) to try it out!")
            print("="*80)
            
        else:
            print("⚠️  No trained models found in S3!")
            print(f"   Location checked: {MODEL_OUTPUT_S3}")
            print(f"\n💡 You need to train a model first:")
            print(f"   1. Stop any running training jobs (Emergency cell)")
            print(f"   2. Run Step 3 (Create estimator)")
            print(f"   3. Run Step 4 (Start training)")
            print(f"   4. Wait 15-20 minutes for training to complete")
            print(f"   5. Then come back here to deploy!")
            
    else:
        print("⚠️  No objects found in model output bucket!")
        print(f"   Location: {MODEL_OUTPUT_S3}")
        print(f"\n💡 Train a model first using Steps 3-4")
        
except Exception as e:
    print(f"❌ Error checking S3: {e}")
    print(f"\n💡 Options:")
    print(f"   1. Train a new model (Steps 3-4)")
    print(f"   2. Check AWS credentials")
    print(f"   3. Verify S3 bucket permissions")

## Step 3: Create HuggingFace Estimator

Configure the training job with the HuggingFace estimator.

## Step 2.5: Split Train/Validation Data (Optional)

If you don't have a separate validation set, run this cell to split your training data (80/20 split).

⚠️ **Run this only once** - it will reorganize your S3 data.

In [ ]:
import random

# Configuration
SPLIT_RATIO = 0.8  # 80% train, 20% validation
SOURCE_PREFIX = 'train/'
TRAIN_PREFIX = 'train_split/'
VAL_PREFIX = 'val/'
CLASSES = ['CollisionImminent', 'EmergencyBraking', 'Normal', 'TireSkid']

print("="*80)
print("🔀 Splitting Training Data into Train/Val Sets")
print("="*80)
print(f"Split ratio: {SPLIT_RATIO*100:.0f}% train, {(1-SPLIT_RATIO)*100:.0f}% validation")
print(f"Source: s3://{bucket_name}/{SOURCE_PREFIX}")
print(f"Train output: s3://{bucket_name}/{TRAIN_PREFIX}")
print(f"Val output: s3://{bucket_name}/{VAL_PREFIX}")
print("\n" + "="*80)

# Initialize counters
total_files = 0
train_count = 0
val_count = 0

# Process each class
for class_name in CLASSES:
    print(f"\n📁 Processing class: {class_name}")
    
    # List all files in this class (with pagination for >1000 files)
    class_prefix = f"{SOURCE_PREFIX}{class_name}/"
    files = []
    paginator = s3.get_paginator('list_objects_v2')
    
    for page in paginator.paginate(Bucket=bucket_name, Prefix=class_prefix):
        if 'Contents' in page:
            # Get all WAV files from this page
            files.extend([obj['Key'] for obj in page['Contents'] if obj['Key'].endswith('.wav')])
    
    if not files:
        print(f"  ⚠️  No WAV files found in {class_prefix}")
        continue
    
    if not files:
        print(f"  ⚠️  No WAV files found in {class_prefix}")
        continue
    
    # Shuffle files for random split
    random.shuffle(files)
    
    # Calculate split point
    split_idx = int(len(files) * SPLIT_RATIO)
    train_files = files[:split_idx]
    val_files = files[split_idx:]
    
    print(f"  Total files: {len(files)}")
    print(f"  Train: {len(train_files)}, Val: {len(val_files)}")
    
    # Copy train files
    for file_key in train_files:
        filename = file_key.split('/')[-1]
        new_key = f"{TRAIN_PREFIX}{class_name}/{filename}"
        s3.copy_object(
            CopySource={'Bucket': bucket_name, 'Key': file_key},
            Bucket=bucket_name,
            Key=new_key
        )
        train_count += 1
    
    # Copy validation files
    for file_key in val_files:
        filename = file_key.split('/')[-1]
        new_key = f"{VAL_PREFIX}{class_name}/{filename}"
        s3.copy_object(
            CopySource={'Bucket': bucket_name, 'Key': file_key},
            Bucket=bucket_name,
            Key=new_key
        )
        val_count += 1
    
    total_files += len(files)
    print(f"  ✓ Copied to train_split/ and val/")

print("\n" + "="*80)
print("✅ Data Split Complete!")
print("="*80)
print(f"Total files processed: {total_files}")
print(f"Train files: {train_count} ({train_count/total_files*100:.1f}%)")
print(f"Validation files: {val_count} ({val_count/total_files*100:.1f}%)")
print("\n📍 New S3 Structure:")
print(f"  Train: s3://{bucket_name}/{TRAIN_PREFIX}")
print(f"  Val: s3://{bucket_name}/{VAL_PREFIX}")
print("\n⚠️  IMPORTANT: Update configuration cell:")
print(f'  TRAIN_S3 = "s3://{bucket_name}/{TRAIN_PREFIX}"')
print(f'  VAL_S3 = "s3://{bucket_name}/{VAL_PREFIX}"')
print("="*80)

In [ ]:
# Create HuggingFace estimator
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='../training',
    role=TRAIN_ROLE_ARN,
    instance_type=TRAIN_INSTANCE_TYPE,
    instance_count=TRAIN_INSTANCE_COUNT,
    transformers_version=TRANSFORMERS_VERSION,
    pytorch_version=PYTORCH_VERSION,
    py_version=PYTHON_VERSION,
    hyperparameters={
        'epochs': EPOCHS,
        'learning-rate': LEARNING_RATE,
        'batch-size': BATCH_SIZE,
        'warmup-steps': WARMUP_STEPS,
        'gradient-accumulation-steps': GRADIENT_ACCUMULATION,
    },
    output_path=MODEL_OUTPUT_S3,
    base_job_name='acousticshield-train',
    sagemaker_session=sagemaker_session,
    disable_profiler=True,  # Disable profiler to reduce overhead
    debugger_hook_config=False,  # Disable debugger to reduce overhead
)

print("✓ HuggingFace estimator created")
print(f"  Base job name: acousticshield-train")
print(f"  Output path: {MODEL_OUTPUT_S3}")

## Step 4: Start Training Job

⏱️ **Expected duration**: 30-40 minutes on ml.g4dn.xlarge

The training job will:
1. Load audio data from S3 using audiofolder format
2. Resample all audio to 16 kHz
3. Extract features using wav2vec2 feature extractor
4. Fine-tune the model for 4 epochs
5. Evaluate on validation set each epoch
6. Save best model based on F1 score
7. Upload model artifacts to S3

In [ ]:
# Prepare training channels
training_channels = {'train': TRAIN_S3}

# Add validation channel if provided
if VAL_S3:
    training_channels['validation'] = VAL_S3
    print(f"📊 Using separate validation set: {VAL_S3}")
else:
    print(f"📊 Validation set will be split from training data (90/10)")

print(f"\n🚀 Starting training job...")
print(f"⏰ Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*80)

# Start training
huggingface_estimator.fit(training_channels, wait=True)

print("\n" + "="*80)
print("✅ Training job completed!")
print(f"⏰ Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📦 Model artifacts: {huggingface_estimator.model_data}")
print("="*80)

## Step 5: Deploy Real-Time Endpoint

⏱️ **Expected duration**: 5-8 minutes

The endpoint will:
- Accept audio/wav input (any sample rate, mono or stereo)
- Auto-resample to 16 kHz if needed
- Return JSON with label, confidence, and probabilities

In [ ]:
# Generate unique endpoint name
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
endpoint_name = f'acousticshield-endpoint-{timestamp}'

print(f"🚀 Deploying endpoint: {endpoint_name}")
print(f"⏰ Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 Instance: {ENDPOINT_INSTANCE_TYPE}")
print("\nThis will take 5-8 minutes...\n")

# Deploy endpoint
predictor = huggingface_estimator.deploy(
    initial_instance_count=ENDPOINT_INSTANCE_COUNT,
    instance_type=ENDPOINT_INSTANCE_TYPE,
    endpoint_name=endpoint_name,
    serializer=DataSerializer(content_type='audio/wav'),
    deserializer=JSONDeserializer(),
)

print("\n" + "="*80)
print("✅ Endpoint deployed successfully!")
print(f"⏰ Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🌐 Endpoint name: {endpoint_name}")
print(f"📍 Status: InService")
print("="*80)

## Step 6: Test Endpoint

### Option A: Test with Sample Audio from S3

In [ ]:
# Download sample audio from S3 for testing
import io

# List available test files
test_bucket = 'acousticshield-ml'
test_prefix = 'train/'  # Or 'val/' if validation set exists

s3 = boto3.client('s3', region_name=region)
response = s3.list_objects_v2(Bucket=test_bucket, Prefix=test_prefix, MaxKeys=10)

if 'Contents' in response:
    # Find first WAV file
    test_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.wav')]
    
    if test_files:
        test_file_key = test_files[0]
        print(f"📁 Using test file: s3://{test_bucket}/{test_file_key}")
        
        # Download file
        wav_buffer = io.BytesIO()
        s3.download_fileobj(test_bucket, test_file_key, wav_buffer)
        wav_bytes = wav_buffer.getvalue()
        
        print(f"✓ Downloaded {len(wav_bytes)} bytes")
    else:
        print("❌ No WAV files found in S3 bucket")
        wav_bytes = None
else:
    print(f"❌ No objects found at s3://{test_bucket}/{test_prefix}")
    wav_bytes = None

### Option B: Generate Synthetic Test Audio

In [ ]:
# Generate synthetic test audio (1 second sine wave at 440 Hz)
import numpy as np
import wave
import struct

print("🎵 Generating synthetic test audio...")

sample_rate = 16000
duration = 1.0
frequency = 440.0  # A4 note

# Generate sine wave
t = np.linspace(0, duration, int(sample_rate * duration))
test_audio = 0.3 * np.sin(2 * np.pi * frequency * t)

# Convert to 16-bit PCM WAV bytes (no soundfile needed!)
wav_buffer = io.BytesIO()

# Write WAV header manually
with wave.open(wav_buffer, 'wb') as wav_file:
    wav_file.setnchannels(1)  # Mono
    wav_file.setsampwidth(2)  # 16-bit
    wav_file.setframerate(sample_rate)
    
    # Convert float audio to 16-bit integers
    audio_int16 = (test_audio * 32767).astype(np.int16)
    
    # Write audio data
    for sample in audio_int16:
        wav_file.writeframes(struct.pack('<h', sample))

wav_bytes = wav_buffer.getvalue()

print(f"✅ Generated {len(wav_bytes)} bytes of test audio")
print(f"   Format: 16 kHz mono WAV, {duration} second sine wave @ {frequency} Hz")
print(f"\n💡 Note: This is just test audio to verify endpoint works")
print(f"   (Not real vehicle sound - just a tone)")

### Invoke Endpoint

In [ ]:
if wav_bytes:
    print("\n🔮 Testing endpoint with format conversion...")
    print("="*80)
    print("⚠️  Note: Pre-trained model needs JSON format, not audio/wav")
    print("   Converting audio to JSON array...\n")
    
    try:
        import librosa
        import numpy as np
        
        # Load and preprocess audio
        print("📊 Step 1: Loading audio...")
        audio, sr = librosa.load(io.BytesIO(wav_bytes), sr=16000, mono=True)
        print(f"   ✅ Loaded: {len(audio)} samples at {sr} Hz")
        
        # Convert to list for JSON
        audio_list = audio.tolist()
        print(f"   ✅ Converted to JSON format")
        
        # Try different payload formats
        print("\n📊 Step 2: Testing endpoint...")
        
        runtime = boto3.client('sagemaker-runtime', region_name=region)
        
        # Try Method 1: Simple inputs
        try:
            print("   🔄 Trying format 1: {\"inputs\": [audio]}...")
            payload = {"inputs": audio_list}
            
            response = runtime.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType='application/json',
                Accept='application/json',
                Body=json.dumps(payload)
            )
            
            result = json.loads(response['Body'].read())
            
            print("   ✅ SUCCESS!\n")
            print("="*80)
            print("📊 PREDICTION RESULTS")
            print("="*80)
            print(json.dumps(result, indent=2))
            
            # Parse and display nicely
            if isinstance(result, list) and len(result) > 0:
                predictions = result[0] if isinstance(result[0], list) else result
                
                print("\n" + "="*80)
                print("🏷️  TOP PREDICTIONS")
                print("="*80)
                
                if isinstance(predictions, list):
                    for i, pred in enumerate(predictions[:5], 1):  # Top 5
                        if isinstance(pred, dict):
                            label = pred.get('label', pred.get('class', 'unknown'))
                            score = pred.get('score', pred.get('confidence', 0))
                            bar = '█' * int(score * 50)
                            print(f"{i}. {label:30s} {score:.2%} {bar}")
                
                print("="*80)
            
            print("\n✅ Endpoint test successful!")
            print("💡 The model is working, but predictions are generic")
            print("   (not trained on your vehicle sound data)")
            
        except Exception as e1:
            print(f"   ❌ Format 1 failed: {str(e1)[:100]}")
            
            # Try Method 2: With parameters
            try:
                print("   🔄 Trying format 2: with sampling_rate...")
                payload = {
                    "inputs": audio_list,
                    "parameters": {"sampling_rate": 16000}
                }
                
                response = runtime.invoke_endpoint(
                    EndpointName=endpoint_name,
                    ContentType='application/json',
                    Accept='application/json',
                    Body=json.dumps(payload)
                )
                
                result = json.loads(response['Body'].read())
                print("   ✅ SUCCESS!\n")
                print("📊 Prediction Results:")
                print(json.dumps(result, indent=2))
                print("\n✅ Endpoint test successful!")
                
            except Exception as e2:
                print(f"   ❌ Format 2 failed: {str(e2)[:100]}")
                print("\n" + "="*80)
                print("❌ ENDPOINT FORMAT INCOMPATIBLE")
                print("="*80)
                print("\n🎯 BOTTOM LINE:")
                print("   The pre-trained model doesn't work well for audio classification.")
                print("\n💡 YOUR BEST OPTION (30 min to working demo):")
                print("   1. Delete this endpoint (Optional cleanup cell)")
                print("   2. Stop old training jobs (Emergency stop cell)")
                print("   3. Train your custom model (Steps 3-4, ~20 min)")
                print("   4. Deploy with your inference.py")
                print("   5. Test with audio/wav - WORKS PERFECTLY!")
                print("\n✅ Benefits of training:")
                print("   • Accepts audio/wav directly (no conversion)")
                print("   • Trained on YOUR vehicle sounds")
                print("   • Recognizes YOUR classes (TireSkid, CollisionImminent, etc.)")
                print("   • Impresses hackathon judges with real ML")
                print("="*80)
            
    except Exception as e:
        print(f"❌ Audio processing failed: {e}")
        print(f"\n💡 Install librosa: pip install librosa")
        
else:
    print("⚠️  No test audio available.")
    print("💡 Run 'Option B: Generate Synthetic Test Audio' cell first")

## Step 7: Test with boto3 SageMaker Runtime (Alternative Method)

This demonstrates how to invoke the endpoint using raw boto3 client.

In [ ]:
if wav_bytes:
    print("🔧 Testing with boto3 SageMaker Runtime client...\n")
    
    # Create SageMaker Runtime client
    runtime_client = boto_session.client('sagemaker-runtime')
    
    # Invoke endpoint
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='audio/wav',
        Accept='application/json',
        Body=wav_bytes
    )
    
    # Parse response
    result = json.loads(response['Body'].read().decode())
    
    print("📊 Response from boto3 client:")
    print(json.dumps(result, indent=2))
    print(f"\n✅ boto3 invocation successful!")
    print(f"   Predicted: {result['label']} ({result['confidence']:.2%})")

## Step 8: Endpoint Information

Save endpoint details for future use.

In [ ]:
print("\n" + "="*80)
print("📋 ENDPOINT INFORMATION")
print("="*80)
print(f"Endpoint Name: {endpoint_name}")
print(f"Region: {region}")
print(f"Instance Type: {ENDPOINT_INSTANCE_TYPE}")
print(f"Instance Count: {ENDPOINT_INSTANCE_COUNT}")
print(f"Model Artifacts: {huggingface_estimator.model_data}")
print(f"\nInput Format: audio/wav (16 kHz mono recommended, auto-resampled)")
print(f"Output Format: application/json")
print(f"\nExpected Output:")
print(f"  {{")
print(f"    \"label\": \"TireSkid\",")
print(f"    \"confidence\": 0.85,")
print(f"    \"probs\": {{")
print(f"      \"Normal\": 0.05,")
print(f"      \"TireSkid\": 0.85,")
print(f"      \"EmergencyBraking\": 0.08,")
print(f"      \"CollisionImminent\": 0.02")
print(f"    }}")
print(f"  }}")
print("="*80)

# Save endpoint info to file
endpoint_info = {
    'endpoint_name': endpoint_name,
    'region': region,
    'instance_type': ENDPOINT_INSTANCE_TYPE,
    'model_artifacts': huggingface_estimator.model_data,
    'created_at': datetime.now().isoformat(),
    'classes': ['Normal', 'TireSkid', 'EmergencyBraking', 'CollisionImminent']
}

with open('endpoint_info.json', 'w') as f:
    json.dump(endpoint_info, f, indent=2)

print("\n✓ Endpoint information saved to endpoint_info.json")

## Optional: Cleanup

⚠️ **WARNING**: This will delete the endpoint. You will be charged while the endpoint is running.

Uncomment and run the cell below to delete the endpoint when done testing.

In [ ]:
# Delete endpoint - uncomment the lines below to clean up
print(f"🗑️  Deleting endpoint: {endpoint_name}")
print(f"⏰ This takes 1-2 minutes...\n")

try:
    predictor.delete_endpoint()
    print("✅ Endpoint deleted successfully!")
    print(f"   Endpoint {endpoint_name} is now removed")
    print(f"\n💰 Cost savings: No more hourly charges!")
    print(f"\n⚠️  Model artifacts remain in S3 and can be redeployed anytime.")
except Exception as e:
    print(f"❌ Error deleting endpoint: {e}")
    print(f"\n💡 Alternative: Delete via AWS Console:")
    print(f"   1. Go to SageMaker → Inference → Endpoints")
    print(f"   2. Find: {endpoint_name}")
    print(f"   3. Click Actions → Delete")

---

## Summary

✅ **Training Complete**: Model fine-tuned on audio classification task  
✅ **Endpoint Deployed**: Real-time inference endpoint is running  
✅ **Testing Complete**: Endpoint responds with predictions  

### Next Steps
1. Integrate endpoint into your application
2. Monitor endpoint metrics in CloudWatch
3. Set up auto-scaling if needed
4. Retrain periodically with new data

### Cost Management
- **Training**: One-time cost (~$0.50-1.00)
- **Endpoint**: Ongoing cost (~$0.23/hour for ml.m5.xlarge)
- **Storage**: Model artifacts in S3 (~$0.02/month)

💡 **Tip**: Delete the endpoint when not in use and redeploy when needed to save costs!

### Documentation
- Endpoint name saved in `endpoint_info.json`
- Model artifacts: `s3://acousticshield-ml/models/`
- Training logs: Available in CloudWatch